<a href="https://colab.research.google.com/github/nadyaloseva/ML_itmo/blob/main/%D0%9C%D0%B0%D1%88_%D0%BB%D0%B0%D0%B1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Аугментация

В команде я занималсь увеличением объема набора данных

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
import numpy as np
import pandas as pd
import torch
from ultralytics import YOLO
import albumentations as A
import os
import yaml
import cv2
from datetime import datetime
from tqdm import tqdm

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Переключаем на свой диск
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import json


file_path = '/content/drive/MyDrive/CNN/Wind Turbine damage/train/_annotations.coco.json'

# Загрузка исходного JSON
with open(file_path) as f:
    coco_data = json.load(f)


print(coco_data)

{'info': {'year': '2025', 'version': '10', 'description': 'Exported from roboflow.com', 'contributor': '', 'url': 'https://app.roboflow.com/datasets/wind-turbines-dar1t/10', 'date_created': '2025-04-20T14:49:17+00:00'}, 'licenses': [{'id': 1, 'url': 'https://creativecommons.org/licenses/by/4.0/', 'name': 'CC BY 4.0'}], 'categories': [{'id': 0, 'name': 'damage', 'supercategory': 'none'}, {'id': 1, 'name': 'corrosion', 'supercategory': 'damage'}, {'id': 2, 'name': 'lightning', 'supercategory': 'damage'}, {'id': 3, 'name': 'lightning receptor', 'supercategory': 'damage'}, {'id': 4, 'name': 'missing teeth', 'supercategory': 'damage'}, {'id': 5, 'name': 'patch', 'supercategory': 'damage'}], 'images': [{'id': 0, 'license': 1, 'file_name': '20240120_155211_mp4-0213_jpg.rf.3f263dc0da2a5ae57494f8d977b901ca.jpg', 'height': 1920, 'width': 1080, 'date_captured': '2025-04-20T14:49:17+00:00', 'extra': {'name': '20240120_155211_mp4-0213.jpg'}}, {'id': 1, 'license': 1, 'file_name': '20240120_155211_mp

In [6]:
# Преобразуем в датафреймы
df_images = pd.DataFrame(coco_data["images"])
df_annots = pd.DataFrame(coco_data["annotations"])
df_cats = pd.DataFrame(coco_data["categories"])

# Объединяем
df = df_annots.merge(df_images, left_on="image_id", right_on="id") \
              .merge(df_cats, left_on="category_id", right_on="id", suffixes=('_img', '_cat'))

In [7]:
df[['file_name', 'id', 'name']].groupby(['name']).count()

,file_name,id
name,,
corrosion,7150,7150
lightning,31,31
lightning receptor,93,93
missing teeth,1630,1630
patch,246,246


Тут мы видим, что количество в разных категориях сильно отличается, поэтому мы сначала пробывали бездумно замножить картинки всех типов коррозии

In [8]:
img_dir = "/content/drive/MyDrive/CNN/Wind Turbine damage/train"
out_img_dir = "/content/drive/MyDrive/CNN/Wind Turbine damage/train_aug"
output_json_path = "/content/drive/MyDrive/CNN/Wind Turbine damage/augmented_train.json"

os.makedirs(out_img_dir, exist_ok=True)

Но после этих попыток решили, что нужно размножать те категории, которых меньше

In [18]:
# Сколько аугментированных копий создавать для каждого класса
'''n_copies = {
    'lightning': 50,
    'lightning receptor': 20,
    'patch': 10,
    'missing teeth': 3,
    'corrosion': 0,
}'''
n_copies = {
    'lightning': 1,
    'lightning receptor': 1,
    'patch': 1,
    'missing teeth': 1,
    'corrosion': 0,
}

In [10]:
# Аугментации
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Rotate(limit=20, p=0.3),
    A.Blur(blur_limit=3, p=0.1),
    A.CLAHE(p=0.1),
], bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']))

In [11]:
# Инициализация новых данных
new_images = []
new_annotations = []

img_id = max(img["id"] for img in coco_data["images"]) + 1
ann_id = max(ann["id"] for ann in coco_data["annotations"]) + 1

In [19]:
# Маппинг
cat_name_to_id = {cat["name"]: cat["id"] for cat in coco_data["categories"]}
cat_id_to_name = {v: k for k, v in cat_name_to_id.items()}

In [20]:
for img_info in tqdm(coco_data["images"]):
    img_file = img_info["file_name"]
    img_path = os.path.join(img_dir, img_file)
    img = cv2.imread(img_path)
    if img is None:
        continue

    anns = [a for a in coco_data["annotations"] if a["image_id"] == img_info["id"]]
    anns_by_class = {}
    for ann in anns:
        cls = cat_id_to_name[ann["category_id"]]
        if cls not in n_copies or n_copies[cls] <= 0:
            continue
        anns_by_class.setdefault(cls, []).append(ann)

    for cls, cls_anns in anns_by_class.items():
        for i in range(n_copies[cls]):
            bboxes = [a["bbox"] for a in cls_anns]
            cat_ids = [a["category_id"] for a in cls_anns]

            try:
                augmented = transform(image=img, bboxes=bboxes, category_ids=cat_ids)
            except Exception as e:
                print(f"Ошибка аугментации {img_file}: {e}")
                continue

            new_img = augmented["image"]
            new_bboxes = augmented["bboxes"]
            new_cat_ids = augmented["category_ids"]

            new_filename = f"{os.path.splitext(img_file)[0]}_{cls}_aug{i}.jpg"
            new_path = os.path.join(out_img_dir, new_filename)
            cv2.imwrite(new_path, new_img)

            new_images.append({
                "id": img_id,
                "file_name": new_filename,
                "width": img_info["width"],
                "height": img_info["height"],
                "date_captured": datetime.utcnow().isoformat(),
                "license": img_info["license"]
            })

            for bbox, cat_id in zip(new_bboxes, new_cat_ids):
                x, y, w, h = bbox
                new_annotations.append({
                    "id": ann_id,
                    "image_id": img_id,
                    "category_id": cat_id,
                    "bbox": [x, y, w, h],
                    "area": w * h,
                    "iscrowd": 0,
                    "segmentation": [],
                })
                ann_id += 1
            img_id += 1

# Объединение с оригиналом
coco_data["images"].extend(new_images)
coco_data["annotations"].extend(new_annotations)

 24%|██▍       | 261/1070 [01:42<06:35,  2.05it/s]

Ошибка аугментации 20240120_163050-0-_jpg.rf.054a5a3a8ca5d5cea97e317a4c6d7883.jpg: Expected y_max for bbox [    0.31183      0.9883     0.31778           1           4] to be in the range [0.0, 1.0], got 1.000026822090149.


 64%|██████▍   | 686/1070 [06:14<04:44,  1.35it/s]

Ошибка аугментации DJI_0389_JPG.rf.dbd07857be83aa42389c66543614d7f0.jpg: Expected x_max for bbox [    0.98106     0.25892      1.0001     0.26768           4] to be in the range [0.0, 1.0], got 1.0000946521759033.


 69%|██████▉   | 741/1070 [06:45<04:03,  1.35it/s]

Ошибка аугментации DJI_0220_JPG.rf.1f887e8da41b0a749cc03a2a6efad969.jpg: Expected y_max for bbox [    0.47557     0.97475     0.47761      1.0001           4] to be in the range [0.0, 1.0], got 1.000074028968811.


100%|██████████| 1070/1070 [10:51<00:00,  1.64it/s]


In [21]:
coco_data["images"].extend(new_images)
coco_data["annotations"].extend(new_annotations)

# Сохранение нового JSON
with open(output_json_path, "w") as f:
    json.dump(coco_data, f)

In [29]:
file_path = '/content/drive/MyDrive/CNN/Wind Turbine damage/train_augmented/_annotations.coco.json'

# Загрузка исходного JSON
with open(file_path) as f:
    coco_data = json.load(f)

In [30]:
# Преобразуем в датафреймы
df_images = pd.DataFrame(coco_data["images"])
df_annots = pd.DataFrame(coco_data["annotations"])
df_cats = pd.DataFrame(coco_data["categories"])

# Объединяем
df = df_annots.merge(df_images, left_on="image_id", right_on="id") \
              .merge(df_cats, left_on="category_id", right_on="id", suffixes=('_img', '_cat'))

In [31]:
df[['file_name', 'id', 'name']].groupby(['name']).count()

,file_name,id
name,,
corrosion,7150,7150
lightning,1545,1545
lightning receptor,1889,1889
missing teeth,3243,3243
patch,2683,2683


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


In [ ]:
# Загрузка модели и перенос на GPU
model = YOLO('yolov8n.pt') .to(device)

100%|██████████| 6.25M/6.25M [00:00<00:00, 115MB/s]


In [ ]:
# Параметры обучения
results = model.train(
    data='/content/drive/MyDrive/CNN/Wind Turbine damage/yolo_wind_turbine_backup/wind_turbine.yaml',
    epochs=50,
    batch=8,
    imgsz=640,
    device='0'  # 0 для GPU, 'cpu' для CPU
)

# Проверка использования GPU
print(f"Модель на GPU: {next(model.parameters()).is_cuda}")  # Это должно вернуть False

Ultralytics 8.3.126 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/CNN/Wind Turbine damage/yolo_wind_turbine_backup/wind_turbine.yaml, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/train3/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nbs=64, nms=False, opset=None, optimize=False, optimizer=au

train: Scanning /content/drive/.shortcut-targets-by-id/1RKiGuI64aK1XUX8lCikv7B-eqpoXtnTy/Wind Turbine damage/yolo_wind_turbine_backup/labels/train.cache... 2107 images, 19 backgrounds, 0 corrupt: 100%|██████████| 2107/2107 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


WARNING ⚠️ val: Slow image access detected (ping: 22.4±42.9 ms, read: 73.5±56.1 MB/s, size: 806.6 KB). Use local storage instead of remote/mounted storage for better performance. See https://docs.ultralytics.com/guides/model-training-tips/


val: Scanning /content/drive/.shortcut-targets-by-id/1RKiGuI64aK1XUX8lCikv7B-eqpoXtnTy/Wind Turbine damage/yolo_wind_turbine_backup/labels/val.cache... 185 images, 86 backgrounds, 0 corrupt: 100%|██████████| 271/271 [00:00<?, ?it/s]


...

                   all        271       2619      0.602      0.201      0.214      0.101



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      28/50      2.78G      2.533      1.538     0.9737          6        640: 100%|██████████| 264/264 [03:40<00:00,  1.20it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:18<00:00,  1.09s/it]


                   all        271       2619      0.664      0.248      0.259      0.136

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      29/50      2.78G      2.541      1.577     0.9977        206        640:  44%|████▍     | 117/264 [02:25<02:15,  1.08it/s]

In [ ]:
# Загружаем лучшую обученную модель
best_model = YOLO(os.path.join('runs', 'detect', 'train4', 'weights', 'best.pt'))

# Валидация
metrics = best_model.val()
print(f"mAP50-95: {metrics.box.map}")
print(f"mAP50: {metrics.box.map50}")
print(f"mAP75: {metrics.box.map75}")

Ultralytics 8.3.126 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.6±0.3 ms, read: 130.3±141.6 MB/s, size: 370.8 KB)


val: Scanning /content/drive/.shortcut-targets-by-id/1RKiGuI64aK1XUX8lCikv7B-eqpoXtnTy/Wind Turbine damage/yolo_wind_turbine_backup/labels/val.cache... 185 images, 86 backgrounds, 0 corrupt: 100%|██████████| 271/271 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:41<00:00,  2.41s/it]


                   all        271       2619      0.444      0.248      0.299       0.15
             corrosion        134       2150      0.239      0.116     0.0825     0.0281
             lightning          9          9          1      0.402      0.737      0.391
    lightning receptor         10         12          0          0          0          0
         missing teeth         81        391      0.329      0.263       0.19     0.0623
                 patch         16         57      0.651      0.456      0.483      0.269
Speed: 0.2ms preprocess, 3.8ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs/detect/val3
mAP50-95: 0.14989323715821803
mAP50: 0.2986022136287486
mAP75: 0.15360644413762503


In [ ]:
import os
import json
import pandas as pd
from ultralytics import YOLO

# Загрузка модели
model = YOLO('runs/detect/train3/weights/best.pt')

# Путь к тестовым изображениям
test_folder = '/content/drive/MyDrive/CNN/Wind Turbine damage/test'

# Собираем результаты
results = []

for img_name in os.listdir(test_folder):
    if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    img_path = os.path.join(test_folder, img_name)
    img = cv2.imread(img_path)
    if img is None:
        continue

    # Делаем предсказание (возвращает список Results объектов)
    predictions = model.predict(img, conf=0.25)

    # Обрабатываем результаты для текущего изображения
    img_objects = []
    for pred in predictions:
        for box in pred.boxes:
            # Получаем данные
            class_id = int(box.cls)
            class_name = model.names[class_id]
            conf = float(box.conf)  # Вероятность (confidence)
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            w = x2 - x1
            h = y2 - y1

            # Добавляем в формате [class, x, y, w, h, confidence]
            img_objects.append([class_name, x1, y1, w, h, conf])

    # Добавляем в общие результаты
    results.append({
        "image_id": img_name,
        "objects": img_objects
    })

# Создаем DataFrame
df_results = pd.DataFrame(results)


0: 640x640 (no detections), 9.1ms
Speed: 10.7ms preprocess, 9.1ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.5ms
Speed: 5.6ms preprocess, 7.5ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.7ms
Speed: 5.4ms preprocess, 7.7ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 13.1ms
Speed: 4.4ms preprocess, 13.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.5ms
Speed: 5.5ms preprocess, 7.5ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.4ms
Speed: 4.5ms preprocess, 7.4ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.4ms
Speed: 4.8ms preprocess, 7.4ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.4ms
Speed: 6.2ms preprocess, 9.4ms inference, 1.1

In [ ]:
df_results['objects'] = df_results['objects'].apply(lambda x: str(x).replace("'", '"'))
df_results

,image_id,objects
0,20240120_153748-0-_jpg.rf.6c50a8089bf3ccd32bb6...,[]
1,20240120_153658_jpg.rf.66dc04dc4ee8194ccc19397...,[]
2,20240120_153711_jpg.rf.3a50aa9daa881abe4296c7f...,[]
3,20240120_150729_jpg.rf.5925f54483de8cc52f75fdd...,[]
4,20240120_153628_jpg.rf.67030e3c4592587d26d7762...,[]
...,...,...
146,DJI_0987_JPG.rf.c2002d99afd301f941c19bd6e56a91...,[]
147,DJI_0993_JPG.rf.f33b6663ee70d58a17f1b41a031a28...,[]
148,DJI_0989_JPG.rf.c88276836792084a0205e542b79fb9...,[]
149,DJI_0994_JPG.rf.e5bf13e77518256c8c3f6ae62f9612...,[]


In [ ]:
df_results.to_csv('predictions.csv', index=False)

In [ ]:
!cp -r /content/runs /content/drive/MyDrive/